In [2]:
import os
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from pandas.core.common import flatten
import copy
import numpy as np
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import glob
from tqdm import tqdm

In [4]:
!pip install kora -q
from kora import drive
drive.link_nbs()

     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 79.3 MB/s 
Mounted at /content/drive


In [ ]:
!pip install import-ipynb
import import_ipynb

In [9]:
import Damage_severity_self_inception


In [10]:
%cd "/gdrive/MyDrive/Colab Notebooks/"



/gdrive/MyDrive/Colab Notebooks


In [16]:
num_filters=16
num_blocks = [5,5,5,5,5]
model_loaded = Damage_severity_self_inception.MyNetwork(num_filters, 'inception', num_blocks)

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
model_path = '/gdrive/MyDrive/project/severity_inception.pt'
# model_loaded = model.load_state_dict(torch.load(model_path))

model_loaded.load_state_dict(torch.load(model_path))
model_loaded.cuda()
model_loaded.eval()

In [22]:
import torchvision.transforms as transforms

def severity_preprocess(img):
  test_transform = transforms.Compose([
    transforms.CenterCrop(128),                       
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

  return test_transform(img)

In [23]:
import urllib

from PIL import Image
from urllib.request import Request, urlretrieve  # Python 3

def severity_test(image_path):
    
    urlretrieve(image_path, 'save.jpg') # or other way to upload image
    # Image.open('save.jpg')
    img = severity_preprocess(Image.open('save.jpg'))
    img = img.to(device)
    img = img.unsqueeze(0)


    with torch.no_grad():
      output = model_loaded(img)
      print(output)
      # print(output)
      if output[0].argmax()==0:
        return ' Minor damage '
      elif output[0].argmax()==1:
        return ' Prediction is: Moderate damage '
      else:
        return ' Prediction is: Severe damage '




In [30]:
url='https://media.istockphoto.com/id/175195079/photo/a-red-car-with-a-damaged-headlight-after-an-accident.jpg?s=612x612&w=0&k=20&c=xM6qXM58jCojDt9wr-g2TRZyBGrL75oy_re2nryRiIw='
print(severity_test(url))


tensor([[-0.3501,  0.7152, -0.0773]], device='cuda:0')
 Prediction is: Moderate damage 
